In [7]:
import os
import psycopg2
import pandas as pd
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from pymilvus import MilvusClient

# ================== Config ==================
DB_PARAMS = {
    "dbname": "video_frame",   # database đã tạo
    "user": "postgres",
    "password": "123",         # đổi theo mật khẩu của bạn
    "host": "localhost",
    "port": "5432"
}

CSV_DIR   = r"D:\Big_project_2025\Video_Similarity_Search\data\csv"
FRAME_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\key_frame"
VIDEO_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\video"
MODEL_DIR = r"D:\Big_project_2025\huggingface_cache"  # nơi lưu model CLIP

# ================== Kết nối DB + Milvus ==================s
conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()
client = MilvusClient(uri="http://localhost:19530")


# Cell 2. Tạo bảng PostgreSQL (chạy 1 lần)
chạy bên pg admin rồi

# Cell 3 – Tạo collection Milvus (chạy 1 lần)

In [9]:
# Kết nối Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")

# Tên collection
collection_name = "text_image_video_collection"

# Xóa collection cũ (nếu có) -> khởi tạo lại từ đầu
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)
    print(f"⚠️ Collection '{collection_name}' đã bị xóa")

# Tạo collection mới
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=512,              # Vector size của CLIP ViT-B/32
    auto_id=True,               # Tự động tạo ID
    enable_dynamic_field=True   # Cho phép thêm field động (vd: frame_path)
)

# Kiểm tra danh sách collection hiện có
collections = milvus_client.list_collections()
print("✅ Danh sách collection:", collections)


✅ Danh sách collection: ['video_search', 'image_collection', 'text_image_video_collection', 'Movies', 'my_rag_collection']


# Cell 4 – Load CLIP model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR)

def encode_image(image_path):
    try:
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features[0].cpu().numpy()
    except Exception as e:
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None


# Cell 5 – Hàm insert tách riêng

In [16]:
# Insert Milvus
def insert_to_milvus(frame_path, emb):
    res = client.insert(
        collection_name="text_image_video_collection",
        data=[{"vector": emb, "frame_path": frame_path}]
    )
    # Trả về ID đầu tiên
    return res["ids"][0]

# Insert PostgreSQL
def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    cur.execute("""
        INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
        VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT (frame_path) DO NOTHING
    """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))


# Cell 6 – Index toàn bộ video (dùng khi mới chạy lần đầu hoặc re-index)

In [ ]:
# ================== Cell 6: Index dữ liệu ==================
import os
import pandas as pd

# Hàm insert vào Milvus
def insert_to_milvus(frame_path, emb):
    try:
        res = client.insert(
            collection_name="text_image_video_collection",
            data=[{"vector": emb, "frame_path": frame_path}]
        )
        # Milvus client v2 trả về dict, primary key nằm trong "ids"
        if "ids" in res and len(res["ids"]) > 0:
            return res["ids"][0]
        return None
    except Exception as e:
        print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
        return None

# Hàm insert vào Postgres
def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    try:
        cur.execute("""
            INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
            VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT (frame_path) DO NOTHING
        """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))
    except Exception as e:
        print(f"❌ Lỗi insert Postgres cho {frame_path}: {e}")

# Hàm index toàn bộ video
def index_videos():
    for key_frame_dir in os.listdir(FRAME_DIR):
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        # Kiểm tra file cần thiết
        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            print(f"⚠️ Thiếu file cho {key_frame_dir}, bỏ qua.")
            continue

        # Đọc mapping CSV
        mapping_df = pd.read_csv(csv_path)

        # Insert video nếu chưa có
        cur.execute("""
            INSERT INTO videos (video_path, title, description)
            VALUES (%s, %s, %s)
            ON CONFLICT (video_path) DO NOTHING
            RETURNING id
        """, (video_path.replace("\\","/"), os.path.basename(video_path), "Video demo"))

        row = cur.fetchone()
        if row:  # nếu insert mới thành công
            video_id = row[0]
        else:    # nếu đã có -> lấy id cũ
            cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path.replace("\\","/"),))
            row = cur.fetchone()
            if row:
                video_id = row[0]
            else:
                print(f"❌ Không tìm thấy video {video_path}, bỏ qua.")
                continue

        # Duyệt từng frame trong thư mục
        for frame_file in os.listdir(frame_dir_path):
            if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                continue
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

            # Lấy index frame từ tên file
            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except:
                continue

            # Lấy row mapping
            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                continue

            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Encode ảnh
            emb = encode_image(frame_path)
            if emb is None:
                continue

            # Insert Milvus
            milvus_id = insert_to_milvus(frame_path, emb)

            # Insert Postgres
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        # Commit sau mỗi video
        conn.commit()
        print(f"✅ Indexed {key_frame_dir}")

# ================== Chạy index ==================
index_videos()


TypeError: 'NoneType' object is not subscriptable

# Cell 7 – Index incremental (chỉ data mới)

In [ ]:
def index_new_data():
    for key_frame_dir in os.listdir(FRAME_DIR):
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            continue

        mapping_df = pd.read_csv(csv_path)

        # Lấy video_id
        cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path,))
        video_id = cur.fetchone()
        if not video_id:
            continue
        video_id = video_id[0]

        # Check frame mới
        for frame_file in os.listdir(frame_dir_path):
            if not frame_file.endswith((".jpg",".jpeg",".png")):
                continue
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

            cur.execute("SELECT 1 FROM frame_mappings WHERE frame_path=%s", (frame_path,))
            if cur.fetchone():  # đã có thì bỏ qua
                continue

            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except:
                continue

            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                continue

            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Encode
            emb = encode_image(frame_path)
            if emb is None:
                continue

            # Insert Milvus + Postgres
            milvus_id = insert_to_milvus(frame_path, emb)
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        conn.commit()
        print(f"✅ Indexed new frames for {key_frame_dir}")


# Với cấu trúc này bạn có thể:

- Chạy Cell 2 + 3 chỉ 1 lần duy nhất để khởi tạo DB và Collection.

- Sau đó dùng Cell 6 để index tất cả data cũ.

- Về sau chỉ cần chạy Cell 7 để nạp thêm data mới vào, không ảnh hưởng dữ liệu cũ.